In [9]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-upswell-vlab', override=True)
db_conf = get_db_conf(env)

In [3]:
USER = "malaria2@vlab.digital"
CAMPAIGN = "mnm-individual-effect"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [56]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [16]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [ ]:
# make creatives conf
from adopt.marketing import CreativeConf, CampaignConf


In [12]:
40000000.0 / 7300

5479.45205479452

In [ ]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 40000000.0,
     'min_budget': 10000.0,
     'opt_window': 48,
     'end_date': '2021-02-14',
     'page_id': '102998371752603',
     'instagram_id': None,
     'proportional': True,
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-individual-effect'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [45]:


with open("config/audiences.json") as f:
    audience_confs = [make_audience_conf(c) for c in json.load(f)]


confs = [dict_from_nested_type(a) for a in audience_confs]

# create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf

def _creative_conf(name, image, body, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "welcome_message": body,
        "link_text": "Messenger main survey shuroo karen!",
        "button_text": "Start!",
        "form": form,
    }

image_confs = [
    ('vlab-mnm-mother-daughter-voice-be-heard', '1.png', 'अपने विचार दुनिया को बताएं!'),
    ('vlab-mnm-family-flowers-keep-healthy', '2.png', 'हमें अपने स्वस्थ जीवन के राज बताएं और फ्री डाटा क्रेडिट जीतें।'),
    ('vlab-mnm-schoolgirl-laptop-got-10-min', '3.png', 'क्या आपके पास सिर्फ १० मिनिट हैं? मोबाइल डाटा क्रेडिट में २०० रुपये पाएँ। आपकी राय हमारे लिए महत्वपूर्ण है!'),
    ('vlab-mnm-grandfather-bike-healthy', '4.png', 'अपने कम्यूनिटी को स्वस्थ रखने में हमें मदद करें- हमें बताएं आप क्या सोचते है!'),
    ('vlab-mnm-woman-sari-texting-got-time', '5.png', 'एक छोटे से सर्वे के लिए आपके पास टाइम है?'),
]

creatives = [CreativeConf(**_creative_conf(name, images[n], body, 'extrabasehin')) for name, n, body in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [43]:
from itertools import product
from functools import reduce

import pandas as pd
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_treatment_targeting(df):
    df = df[df.treatment == df.treatment.iloc[0]]
    ids = df.disthash.unique()

    vars_ = [{'op': 'equal', 
              'vars': [{'type': 'response', 'value': 'md:stratumid'}, 
                       {'type': 'constant', 'value': i}]}
             for i in ids]

    return {'op': 'or', 'vars': vars_}


def format_group(g):
    kw, fb, qu = zip(*g)

    combine = lambda a, b: {**a, **b}
    kw = reduce(combine, kw)
    fb = reduce(combine, fb)
    return {**kw,
            'id_': f'kutcha-{int(kw["kutcha"])}-treatment-{kw["treatment"]}-gender-{fb["genders"][0]}-age-{fb["age_min"]}',
            'quota': 100,
            'facebook_targeting': fb,
            'question_targeting': [q for q in qu if q]}



def make_stratum(df, id_, kutcha, treatment, quota, facebook_targeting, question_targeting):
    df = df[df.treatment == treatment].reset_index(drop=True)

    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    # Deal with audiences for kutcha...    
    excluded_audiences = ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-round-B"]
    audiences = []
    if kutcha:
        audiences = ["vlab-mnm-kutcha-lookalike"]
    else:
        excluded_audiences += ["vlab-mnm-kutcha-lookalike"]


    # TODO: add and OR
    # such that it is EITHER treatment_targeting OR stratumid
    # otherwise new recruits, for which we don't have their district,
    # they do not fall into any category here...
    
    # {'op': 'or', 
    #  'vars': [
    #      {'op': 'equal',
    #       'vars': [{'type': 'response', 'value': 'md:stratumid'}, 
    #                {'type': 'constant', 'value': id_}]},
    #      make_treatment_targeting(df)
    #  ]}
    question_targeting = {'op': 'and', 
                          'vars': [make_treatment_targeting(df), 
                                   *question_targeting]}
    conf = { 
        'id': id_,
        'quota': quota,
        'audiences': audiences,
        'excluded_audiences': excluded_audiences,
        'metadata': {'stratumid': id_},
        'facebook_targeting': targeting, 
        'question_targeting': question_targeting,
        'creatives': ['vlab-mnm-mother-daughter-voice-be-heard',
                      'vlab-mnm-family-flowers-keep-healthy',
                      'vlab-mnm-schoolgirl-laptop-got-10-min',
                      'vlab-mnm-grandfather-bike-healthy',
                      'vlab-mnm-woman-sari-texting-got-time'],
        'shortcodes': ['extrabasehin'],}

    return conf



In [44]:
groups = [
    [
        ({}, {'genders': [1]}, {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'gender'}, 
                                                    {'type': 'constant', 'value': 'Male'}]}), 
        ({}, {'genders': [2]}, {'op': 'equal', 'vars': [{'type': 'translated_response', 'value': 'gender'}, 
                                                    {'type': 'constant', 'value': 'Female'}]}),
    ],
    [
        ({}, {'age_min': 20, 'age_max': 25}, 
         {'op': 'less_than', 'vars': [{'type': 'response', 'value': 'age'},
                                      {'type': 'constant', 'value': 26}]}), 

        ({}, {'age_min': 26, 'age_max': 39}, 
         {'op': 'and', 'vars': [
             {'op': 'greater_than', 'vars': [{'type': 'response', 'value': 'age'},
                                            {'type': 'constant', 'value': 25}]}, 
             {'op': 'less_than', 'vars': [{'type': 'response', 'value': 'age'},
                                         {'type': 'constant', 'value': 40}]}]}),

        ({}, {'age_min': 40}, 
         {'op': 'greater_than', 'vars': [{'type': 'response', 'value': 'age'},
                                         {'type': 'constant', 'value': 39}]}), 
    ],
    [
        ({'treatment': 1}, {}, {}),
        ({'treatment': 0}, {}, {}),
    ],
    [
        ({'kutcha': True}, {}, { 'op': 'equal',
                                 'vars': [{'type': 'response', 'value': 'dwelling'},
                                          {'type': 'constant', 'value': 'A'}]}),
        ({'kutcha': False}, {},  { 'op': 'not_equal',
                                 'vars': [{'type': 'response', 'value': 'dwelling'},
                                          {'type': 'constant', 'value': 'A'}]}),
    ]    
]

treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
df = load_cities('output/cities.csv')
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]
df = df.merge(treatment_assignment)

confs = [format_group(g) for g in product(*groups)]
stratum_confs = [make_stratum(df, **c) for c in confs]
stratum_confs = [dict_from_nested_type(make_stratum_conf(c)) for c in stratum_confs]

In [ ]:
create_campaign_confs(CAMPAIGNID, "stratum", stratum_confs, db_conf)

# TESTING

In [6]:
mal = load_basics(CAMPAIGNID, env)

In [7]:
from adopt.malaria import update_audience_for_campaign, update_ads_for_campaign

# instructions = update_audience_for_campaign(mal)
instructions, report = update_ads_for_campaign(mal)

/home/jupyter/.local/lib/python3.8/site-packages/facebook_business/utils/api_utils.py:30: UserWarning: Value of optimization_goal is not be compatible. Expect OptimizationGoal; got <class 'str'>
  warnings.warn(message)
INFO:root:Loaded 24 adsets from campaign impact-evaluation-individual-effect


INFO:root:Falling back to base budget due to the follow error: Could not calculate the price of any adset between 2021-01-30 and 2021-02-01


INFO:root:Loaded 47 ads from campaign impact-evaluation-individual-effect


In [9]:
len(instructions)

97

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [31]:
from adopt.malaria import hydrate_strata, hydrate_audiences

In [ ]:
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)